### SQLite 数据类型

SQLite 采用动态类型系统，这意味着数据类型与值本身相关联，而不是与存储值的容器（列）相关联。尽管如此，为了与其他 SQL 数据库引擎兼容，SQLite 使用“类型亲和性”的概念。类型亲和性是指为列中存储的数据推荐的类型。

**SQLite 主要有五种存储类别：**

* **NULL**：表示空值。
* **INTEGER**：表示整数值。布尔值也存储为整数（0 表示 false，1 表示 true）。
* **REAL**：表示浮点数值。
* **TEXT**：表示文本字符串。
* **BLOB** (Binary Large Object)：用于存储二进制大对象，例如图像或文件，按输入原样存储。

**类型亲和性：**

SQLite 中的每一列都会被分配以下五种类型亲和性之一：

* **TEXT**：具有 TEXT 亲和性的列会存储所有数据为 NULL、TEXT 或 BLOB 类型。
* **NUMERIC**：具有 NUMERIC 亲和性的列可以存储所有五种存储类别的数据。在可能的情况下，它会将文本转换为 INTEGER 或 REAL。
* **INTEGER**：行为与 NUMERIC 类似。
* **REAL**：具有 REAL 亲和性的列会将整数值强制转换为浮点表示。
* **BLOB**：具有 BLOB 亲和性的列不偏好任何存储类别，也不会尝试强制转换数据。

**日期和时间：**

SQLite 没有单独的日期和时间存储类别。相反，它使用内置函数将日期和时间存储为以下三种格式之一：

* **TEXT**：ISO8601 格式的字符串。
* **REAL**：儒略日数。
* **INTEGER**：Unix 时间戳。

在查询执行期间，SQLite 可能会在数字存储类别（INTEGER 和 REAL）与 TEXT 之间转换值。


sqlite-vss（SQLite Vector Similarity Search，向量相似度搜索）是一个基于Faiss的SQLite扩展，为SQLite带来向量搜索功能。它可以用于构建语义搜索引擎、推荐系统或问答工具。

- 底层Faiss索引的大小上限为1GB。关注[#1](https://github.com/asg017/sqlite-vss/issues/1)获取更新。
- 目前不支持在KNN搜索之上进行额外的过滤。关注[#2](https://github.com/asg017/sqlite-vss/issues/2)获取更新。
- 仅支持CPU Faiss索引，尚不支持GPU。关注[#3](https://github.com/asg017/sqlite-vss/issues/3)获取更新。
- 尚不支持内存映射(mmap)索引，因此索引必须适合RAM。关注[#4](https://github.com/asg017/sqlite-vss/issues/4)获取更新。
- 此扩展使用C++编写，尚无模糊测试。关注[#5](https://github.com/asg017/sqlite-vss/issues/5)获取更新。
- vss0虚拟表不支持`UPDATE`语句，但支持`INSERT`和`DELETE`语句。关注[#7](https://github.com/asg017/sqlite-vss/issues/7)获取更新。



sqlite-vss的开发和采用存在许多障碍，包括：

只能在Linux + MacOS机器上工作（不支持Windows、WASM、移动设备等）
将所有向量都存储在内存中
各种与事务相关的bug和问题
编译极其困难且耗时
缺少常见的向量操作（标量/二进制量化）
几乎所有这些问题都是因为sqlite-vss依赖于Faiss。花费大量时间和精力，其中一些问题也许可以解决，但许多问题会被Faiss阻碍。

考虑到这一点，一个无依赖和低级别的解决方案看起来非常诱人。事实证明，向量搜索并不太复杂，所以sqlite-vec诞生了！

sqlite-vec将会是什么
sqlite-vec将是一个纯C语言编写、无依赖的SQLite扩展。它将提供自定义SQL函数和虚拟表用于快速向量搜索，以及其他处理向量的工具和实用程序（量化、JSON/BLOB/numpy转换、向量运算等）。

一个极小、"足够快"的向量搜索 SQLite 扩展，可在任何地方运行！是 sqlite-vss 的接班人。

在 vec0 虚拟表中存储和查询浮点、int8 和二进制向量
使用纯 C 语言编写，无依赖，可在任何支持 SQLite 的地方运行
（Linux/MacOS/Windows，浏览器 WASM，树莓派等）
在元数据、辅助或分区键列中存储非向量数据

In [1]:
import sqlite3
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
import os
import json
import pickle
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

c:\Users\k\.conda\envs\SmartImageFinder\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
try:
    # 首先尝试加载本地模型
    model_path = r'C:\Users\k\Desktop\BaiduSyncdisk\baidu_sync_documents\hf_models\bge-m3'
    model = SentenceTransformer(model_path)
    print(f"成功加载本地模型: {model_path}")
except Exception as e:
    # 如果本地模型加载失败，使用Hugging Face上的小型模型
    print(f"本地模型加载失败: {e}")
    print("尝试从Hugging Face加载多语言模型...")
    model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
    print("成功加载Hugging Face模型")

# 获取模型的输出维度
embedding_dim = model.get_sentence_embedding_dimension()
print(f"模型输出维度: {embedding_dim}")

成功加载本地模型: C:\Users\k\Desktop\BaiduSyncdisk\baidu_sync_documents\hf_models\bge-m3
模型输出维度: 1024


In [3]:
# 创建一个SQLite数据库连接
db_path = "vector_demo.db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()


In [4]:
# 尝试加载sqlite-vec扩展
try:
    conn.enable_load_extension(True)
    # 根据操作系统加载不同的扩展文件
    conn.execute("SELECT load_extension('./vec0.dll')")
    # 测试扩展是否成功加载
    cursor.execute("SELECT vec_version()")
    version = cursor.fetchone()[0]
    print(f"成功加载sqlite-vec扩展，版本: {version}")
except Exception as e:
    print(f"加载sqlite-vec扩展失败: {e}")
    print("将使用纯Python实现向量操作")
    print("如需使用sqlite-vec扩展，请从 https://github.com/asg017/sqlite-vec/releases 下载对应版本")

成功加载sqlite-vec扩展，版本: v0.1.7-alpha.2


In [5]:
# 创建一个SQLite数据库连接
no_vec0_db_path = "vec_no_vec0.db"
no_vec0_conn = sqlite3.connect(no_vec0_db_path)
no_vec0_cursor = no_vec0_conn.cursor()


In [6]:
# 删除原来的表，如果存在
cursor.execute("DROP TABLE IF EXISTS vec_documents;")
# 创建表格
cursor.execute(f"""
                CREATE VIRTUAL TABLE IF NOT EXISTS vec_documents USING vec0(
                    document_id INTEGER PRIMARY KEY,
                    content_embedding FLOAT[{embedding_dim}] DISTANCE_METRIC=cosine,
                    category TEXT,
                    +original_content TEXT
                );
                """)

In [7]:
no_vec0_cursor.execute(f"""
                    CREATE VIRTUAL TABLE IF NOT EXISTS vec_documents USING vec0(
                        document_id INTEGER PRIMARY KEY,
                        content_embedding FLOAT[{embedding_dim}] DISTANCE_METRIC=cosine,
                        category TEXT,
                        +original_content TEXT
                    );
                    """)
# 会报错，因为没有加载sqlite-vec扩展，OperationalError: no such module: vec0


In [ ]:
# 重新创建一个表：
# 1. 删除原有表
no_vec0_cursor.execute("DROP TABLE IF EXISTS vec_documents;")
# 2. 创建新表
no_vec0_cursor.execute(f"""
                    CREATE TABLE IF NOT EXISTS vec_documents(
                        document_id INTEGER PRIMARY KEY,
                        content_embedding BLOB,
                        category TEXT,
                        original_content TEXT
                    );
                    """)

In [9]:
def no_vec0_insert_documents(cursor, documents):
    """插入文档"""
    # 清空表
    cursor.execute("DELETE FROM vec_documents")
    
    for doc in documents:
        # 标准表插入
        embedding_bytes = pickle.dumps(doc["embedding"])
        cursor.execute(
            "INSERT INTO vec_documents(document_id, content_embedding, category, original_content) VALUES (?, ?, ?, ?)",
            (doc["id"],  embedding_bytes,doc["category"], doc["content"],)
        )
    
    conn.commit()
    print(f"成功插入 {len(documents)} 条文档")

def insert_documents(cursor, documents):
    # 清空表
    cursor.execute("DELETE FROM vec_documents")
    for doc in documents:
        # 虚拟表插入（如果可用）
        embedding_json = json.dumps(doc["embedding"].tolist())
        try:
            cursor.execute(
                "INSERT INTO vec_documents(document_id, content_embedding, category, original_content) VALUES (?, vec_f32(?), ?, ?)",
                (doc["id"], embedding_json, doc["category"], doc["content"])
            )
        except Exception as e:
            print(f"插入虚拟表失败: {e}")

In [10]:
# 准备一些示例文本
documents = [
    {"id": 1, "content": "机器学习是人工智能的一个子领域", "category": "技术"},
    {"id": 2, "content": "深度学习是机器学习的一种方法", "category": "技术"},
    {"id": 3, "content": "向量数据库可以高效存储和检索向量数据", "category": "数据库"},
    {"id": 4, "content": "SQLite是一个轻量级的关系型数据库", "category": "数据库"},
    {"id": 5, "content": "Python是一种流行的编程语言", "category": "编程"},
    {"id": 6, "content": "自然语言处理是处理人类语言的技术", "category": "技术"},
    {"id": 7, "content": "向量相似度搜索在推荐系统中很常用", "category": "技术"},
    {"id": 8, "content": "大数据分析需要高效的数据存储和处理", "category": "数据"}
]

# 为每个文档生成embedding向量
for doc in documents:
    embedding = model.encode(doc["content"])
    doc["embedding"] = embedding

# 展示部分数据
for doc in documents[:2]:
    print(f"ID: {doc['id']}, 内容: {doc['content']}")
    print(f"向量维度: {len(doc['embedding'])}, 向量前几个元素: {doc['embedding'][:5]}...\n")

ID: 1, 内容: 机器学习是人工智能的一个子领域
向量维度: 1024, 向量前几个元素: [-0.03649624 -0.02476022 -0.04679906 -0.00498728  0.00767139]...

ID: 2, 内容: 深度学习是机器学习的一种方法
向量维度: 1024, 向量前几个元素: [-0.02456879 -0.06428009 -0.04240257 -0.00734466 -0.02539387]...



In [11]:
# 这里是使用向量浮点数插入的
insert_documents(cursor,documents)

In [12]:
# 这里使用没有扩展的插入浮点数向量的库就会导致报错
insert_documents(no_vec0_cursor, documents)
# 插入虚拟表失败: no such function: vec_f32
# 换回no_vec0_insert_documents
no_vec0_insert_documents(no_vec0_cursor, documents)

OperationalError: database is locked

In [15]:
def knn_search(cursor, query_embedding, k=3, category=None):
    """执行KNN搜索"""
        # 使用vec0虚拟表

    query_json = json.dumps(query_embedding.tolist())
    if category:
        cursor.execute("""
        SELECT document_id, original_content, category, distance
        FROM vec_documents 
        WHERE content_embedding MATCH ? AND k = ? AND category = ?
        """, (query_json, k, category))
    else:
        cursor.execute("""
        SELECT document_id, original_content, category, distance
        FROM vec_documents 
        WHERE content_embedding MATCH ? AND k = ?
        """, (query_json, k))
    
    return [(row[0], row[1], row[2], row[3]) for row in cursor.fetchall()]


def no_vec0_knn_search(cursor, query_embedding, k=3, category=None):
    """执行KNN搜索"""
        # 使用vec0虚拟表

    query_json = json.dumps(query_embedding.tolist())
    if category:
        cursor.execute("""
        SELECT document_id, original_content, category
        FROM vec_documents 
        WHERE content_embedding MATCH ? AND k = ? AND category = ?
        """, (query_json, k, category))
    else:
        cursor.execute("""
        SELECT document_id, original_content, category
        FROM vec_documents 
        WHERE content_embedding MATCH ? AND k = ?
        """, (query_json, k))
    
    return [(row[0], row[1], row[2], row[3]) for row in cursor.fetchall()]


In [16]:
# 生成查询向量
query_text = "数据库技术与应用"
query_embedding = model.encode(query_text)


In [17]:

# 执行KNN查询
k = 3
results = knn_search(cursor,query_embedding, k)

print(f"查询：'{query_text}'的最近{k}个结果:")
for doc_id, content, category, distance in results:
    print(f"ID: {doc_id}, 距离: {distance:.4f}, 类别: {category}, 内容: {content}")

查询：'数据库技术与应用'的最近3个结果:
ID: 3, 距离: 0.4323, 类别: 数据库, 内容: 向量数据库可以高效存储和检索向量数据
ID: 8, 距离: 0.4413, 类别: 数据, 内容: 大数据分析需要高效的数据存储和处理
ID: 4, 距离: 0.4911, 类别: 数据库, 内容: SQLite是一个轻量级的关系型数据库


可以手动使用vec_distance_L2()、vec_distance_L1()或vec_distance_cosine()，并通过ORDER BY子句执行暴力KNN查询。
select
  id,
  contents,
  vec_distance_L2(contents_embedding, '[2.2, 2.2, 2.2, 2.2]') as distance
from documents
order by distance;


In [18]:
# 试试使用vec_distance_cosine函数
def knn_with_cosine_distance(cursor, query_embedding, k=3, category=None):
    """执行KNN搜索"""
    # 使用vec0虚拟表
    query_json = json.dumps(query_embedding.tolist())
    if category:
        cursor.execute("""
        SELECT document_id, original_content, category, vec_distance_cosine(content_embedding, ?) AS distance
        FROM vec_documents 
        WHERE content_embedding MATCH ? AND k = ? AND category = ?
        """, (query_json, query_json, k, category))
    else:
        cursor.execute("""
        SELECT document_id, original_content, category, vec_distance_cosine(content_embedding, ?) AS distance
        FROM vec_documents 
        WHERE content_embedding MATCH ? AND k = ?
        """, (query_json, query_json, k))
    
    return [(row[0], row[1], row[2], row[3]) for row in cursor.fetchall()]

In [19]:
# 试试knn_with_cosine_distance
results = knn_with_cosine_distance(cursor, query_embedding, k)
print(f"查询：'{query_text}'的最近{k}个结果（使用vec_distance_cosine函数）:")
for doc_id, content, category, distance in results:
    print(f"ID: {doc_id}, 距离: {distance:.4f}, 类别: {category}, 内容: {content}")

查询：'数据库技术与应用'的最近3个结果（使用vec_distance_cosine函数）:
ID: 3, 距离: 0.4323, 类别: 数据库, 内容: 向量数据库可以高效存储和检索向量数据
ID: 8, 距离: 0.4413, 类别: 数据, 内容: 大数据分析需要高效的数据存储和处理
ID: 4, 距离: 0.4911, 类别: 数据库, 内容: SQLite是一个轻量级的关系型数据库


In [ ]:

# 执行KNN查询
k = 3

results = knn_search(no_vec0_cursor,query_embedding, k)
# 会报错，因为没有插件OperationalError: no such column: distance

print(f"查询：'{query_text}'的最近{k}个结果:")
for doc_id, content, category, distance in results:
    print(f"ID: {doc_id}, 距离: {distance:.4f}, 类别: {category}, 内容: {content}")

OperationalError: no such column: distance

In [ ]:
# 执行KNN查询
k = 3
# 我们继续试试没有distance的查询，同样报错：OperationalError: no such column: k，
# 其实是由于没有插件的时候match的执行顺序已经变了，
# 所以导致了没有distance的查询，但是我们删掉了distance，还是存在其他的错误，比如不支持k参数等等，所以我们可以手动实现一个
results = no_vec0_knn_search(no_vec0_cursor,query_embedding, k)
print(f"查询：'{query_text}'的最近{k}个结果:")
for doc_id, content, category in results:
    print(f"ID: {doc_id}, 类别: {category}, 内容: {content}")

OperationalError: no such column: k

In [ ]:
def no_vec0_knn_search(cursor, query_embedding, k=3, category=None):
    """手动实现KNN搜索，针对没有vec0插件的情况"""
    results = []
    
    # 构建查询SQL
    if category:
        query = "SELECT document_id, original_content, category, content_embedding FROM vec_documents WHERE category = ?"
        cursor.execute(query, (category,))
    else:
        query = "SELECT document_id, original_content, category, content_embedding FROM vec_documents"
        cursor.execute(query)
    
    # 处理每一行数据
    for row in cursor.fetchall():
        doc_id, content, doc_category, embedding_bytes = row
        try:
        
            # 使用pickle反序列化embedding
            doc_embedding = pickle.loads(embedding_bytes)
            
            # 计算余弦相似度
            similarity = cosine_similarity([query_embedding], [doc_embedding])[0][0]
            # 计算距离（1 - 相似度，使得值越小越相似）
            distance = 1 - similarity
            results.append((doc_id, content, doc_category, distance))
        except Exception as e:
            print(f"Error processing embedding for doc_id {doc_id}: {e}")
            continue
    
    # 排序并返回前k个结果
    results.sort(key=lambda x: x[3])  # 按距离排序
    return results[:k]


In [ ]:
# 重新试试检索
results = no_vec0_knn_search(no_vec0_cursor,query_embedding, k)
print(f"查询：'{query_text}'的最近{k}个结果:")
for doc_id, content, category, distance in results:
    print(f"ID: {doc_id}, 距离: {distance:.4f}, 类别: {category}, 内容: {content}")


查询：'数据库技术与应用'的最近3个结果:
ID: 3, 距离: 0.4323, 类别: 数据库, 内容: 向量数据库可以高效存储和检索向量数据
ID: 8, 距离: 0.4413, 类别: 数据, 内容: 大数据分析需要高效的数据存储和处理
ID: 4, 距离: 0.4911, 类别: 数据库, 内容: SQLite是一个轻量级的关系型数据库


1. <https://alexgarcia.xyz/blog/2024/building-new-vector-search-sqlite/index.html>
2. <https://github.com/asg017/sqlite-vss/blob/main/README.md>
3. <https://github.com/asg017/sqlite-vec/tree/main>
4. <https://alexgarcia.xyz/sqlite-vec/features/knn.html>
